In [1]:
import spotipy
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import re
import math
from tqdm import tqdm

load_dotenv('../secrets/.env', override=True)

%run useful_functions.ipynb
%run spotify_scraping.ipynb

scope = "playlist-modify-private"

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

my_id = spotify.current_user()['id']

In [2]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

tracks = get_spotify_tracks()
albums = get_spotify_albums()
artists = get_spotify_artists()
search = get_spotify_search()
playlists = get_spotify_playlists()
playlist_tracks = get_spotify_playlist_tracks()

master_df = (
broadcast_playlists
.merge(broadcast_details, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
.merge(shows, on='Show Key', how='left')
.merge(search,left_on=['Artist Name','Track Name'], right_on=['Original Search Artist','Original Search Track'], how = "left", suffixes=("", " Search"))
.merge(tracks, on='Track ID', how='left', suffixes=("", " Tracks"))
.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
)

del(shows,
    show_broadcast_details,
    broadcast_details,
    broadcast_playlists,
    search)

In [3]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year) == 2022) & (master_df['Show Name'] == "Radio 1's Indie Show with Jack Saunders")].drop_duplicates(subset=['Track ID'], keep='first')

In [32]:
def edit_playlist_tracks(playlist_id, new_tracks, user = my_id):
    get_playlist(playlist_id = playlist_id, with_tracks = True)
    current_playlist = playlist_tracks[playlist_tracks['Playlist ID'] == playlist_id]
    return(current_playlist)

edit_playlist_tracks(x, [])

,Track ID,Playlist ID,Position Number
0,0UgCI5TiOQthbrEVSqHC9j,5uIyi7NxggSq80w115PoDQ,0
0,75FTM8KrqDET0dk4fn4xcM,5uIyi7NxggSq80w115PoDQ,1
0,4MP5tSTc6LxY8xhrUOv9Ya,5uIyi7NxggSq80w115PoDQ,2
0,15EgMjlIMSbRlutS6v8EPI,5uIyi7NxggSq80w115PoDQ,3
0,2venvrFxLv6wGWmZwQdGYg,5uIyi7NxggSq80w115PoDQ,4
...,...,...,...
0,2mdaMDbjFxmxQPO90PxT5w,5uIyi7NxggSq80w115PoDQ,694
0,2JjDl1YbzaBKlhSrUXZqNM,5uIyi7NxggSq80w115PoDQ,695
0,0AIlxnJ1kFpu6wp89rjjUv,5uIyi7NxggSq80w115PoDQ,696
0,7BZWTu4Pv77TxGmmsQaSXN,5uIyi7NxggSq80w115PoDQ,697


In [20]:
spotify.playlist_add_items(playlist_id, [temp_tracks_to_add[0]], position=0)

NameError: name 'playlist_id' is not defined

In [29]:
playlists = pd.DataFrame()
playlist_tracks = pd.DataFrame()

#make_playlist("Test Playlist","test description")
get_playlist('5uIyi7NxggSq80w115PoDQ', with_tracks=False)
x = playlists[playlists['name'] == 'Indie Show 2022']['id'][0]
edit_playlist_details(user=my_id, playlist_id = x, name='Indie Show 2022', description='If this is working and updating by itself, then my python coding skills are better than I think :)')
get_playlist('5uIyi7NxggSq80w115PoDQ')

#delete_playlist(playlist_id=x)

playlist_tracks.to_pickle('../data/spotify_playlist_tracks.pkl')
playlists.to_pickle('../data/spotify_playlists.pkl')

In [155]:
#rhead = artists[artists['Artist Name'] == 'Radiohead']['Artist ID'][0]
#rh_tracks = tracks[tracks["Artist ID's"].apply(lambda x: True if (rhead in x) else False)]

#spotify.audio_analysis("4u7EnebtmKWzUH433cf5Qv")
#spotify.audio_features("4u7EnebtmKWzUH433cf5Qv")
#spotify.recommendation_genre_seeds()

#set(master_df['Show Name'])
#set(demo_df['Service Date'])

In [ ]:
def show_tracks(tracks):
    for i, item in enumerate(tracks['items']):
        track = item['track']
        print("   %d %32.32s %s" % (i, track['artists'][0]['name'],
            track['name']))



        playlists = spotify.user_playlists(username)
        for playlist in playlists['items']:
            if playlist['owner']['id'] == username:
                print()
                print(playlist['name'])
                print ('  total tracks', playlist['tracks']['total'])
                results = sp.user_playlist(username, playlist['id'],
                    fields="tracks,next")
                tracks = results['tracks']
                show_tracks(tracks)
                while tracks['next']:
                    tracks = sp.next(tracks)
                    show_tracks(tracks)
    else:
        print("Can't get token for", username)

In [50]:
test_playlist = spotify.user_playlist_create(my_id, "Future Artists Show 2022", public=False, collaborative=False, description='If this is working and updating by itself, then my python coding skills are better than I think :)')
playlist_id = test_playlist['id']
tracks_to_add = [x for x in demo_df['Track ID'] if isinstance(x,str) == True]
i = 0
while i < len(tracks_to_add) // 50 + 1:
    temp_tracks_to_add = tracks_to_add[50*i:50*(i+1)]
    spotify.user_playlist_add_tracks(my_id, playlist_id, temp_tracks_to_add, position=50*i)
    i += 1